In [111]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os
import sys
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Bar
init_notebook_mode(connected=True)

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from analyser import Analyser
from db import SqliteDb as db

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Documentation

## Research Questions

** Which contribution have bots to the quality of Wikidata? **
* Which differences exist between bots which successfully completed the request for permission process and those who doesn't? (Hypothesis: The request for permission process is a quality gate for bots)
* How does bot's field of activity change over time?
* What is the difference between a bot with the group 'bot' and a bot with a botflag?

## Data acquisition and preparation

### Requests for permissions

As a first step we downloaded and parsed the data about requests for permissions about bots from Wikidata in July 2018. We collected the already closed requests which can be found in the Wikidata archive (e.g. https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Archive#Requests_for_bot_flags). The data is stored in the table `requests_for_permissions` database.

Request for permissions which have no own page were left out (e.g. https://www.wikidata.org/w/index.php?title=Wikidata:Requests_for_permissions/Bot/Checkallthestrings_bot_1&action=edit&redlink=1).

Request for permissions which are listed several times in different archives were only parsed once (e.g. https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Bot/VIAFbot is listes in https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/March_2013 and in https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/April_2013).

### Bots
In a second step we fetched data from five different sources that contain lists of bots or information that can help to detect bots. None of the list is complete. The following table lists these sources and describe what they contain. The data is first saved in csv files (also listed in the table) and afterwards stored in the table `bots` in the database.

| description                           |  source               | output file          |
| ---------------------------------- |:--------------------:|:--------------------:|
| users in the group 'bot' | MediaWiki API (Example URL: https://www.wikidata.org/w/api.php?action=query&list=allusers&augroup=bot) | data/parser/users_in_bot_group.csv |
| bots that have a closed request for permission | [Requests for bot flags](https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Archive#Requests_for_bot_flags ) | data/spiders/bots_with_requests_for_permissions.csv |
| bots with a botflag     | [Bots with botflag](https://www.wikidata.org/wiki/Category:Bots_with_botflag) | data/spiders/bots_with_botflag.csv |
| bots without a botflag  | [Bots without a botflag](https://www.wikidata.org/wiki/Category:Bots_without_botflag) | data/spiders/bots_without_botflag.csv |
| extension bots           | [Extension bots](https://www.wikidata.org/wiki/Category:Extension_bots) | data/spiders/extension_bots.csv |

### Database

#### Schema
The original db schema can be found under the path `db/schema.ddl`. The db schema as diagram looks as follows: 

Note: The lates update for this diagram was on 17 July 2018 and may not be up-to-date!

![DB schema](images/db_schema.png)

#### Tables

The tabel `requests_for_permissions` contains the following content: 

Note: The lates update for this table was on 17 July 2018 and thus may not be up-to-date!

| column                | content explanation                                                                   |
|:---------------------:|:-------------------------------------------------------------------------------------:|
| id                    | unique identifier                                                                     |
| url                   | url of the request for permission                                                     |
| bot_url               | url of the bot for which the request for permission is for                            |
| bot_name              | name of the bot for which the request for permission is for                           |
| bot_has_red_link      | flag which states if the url of the bot is a red link (i.e. not working)              |
| operator_url          | url of the operator which opend the request for permission                            |
| operator_name         | name of the operator which opend the request for permission                           |
| operator_has_red_link | flag which states if the url of the operator is a red link (i.e. not working)         |
| is_successful         | flag which states if the request for permission was successful. 1 if it was and 0 if not     |
| first_edit            | timestamp when the first edit of this request for permission was made         |
| last_edit             | timestamp when the last edit of this request for permission was made          |
| closed_at             | timestamp when the request for permission was closed which is equvalent to the time when the summary was written |
| revision_count        | count how many revisions were applied to the request for permission                   |
| editor_count          | count how many different editors made revisons to the request for permission          |
| comment_symbol_count  | count how often a comment symbol was used in the discussion of the request            |
| question_symbol_count | count how often a question symbol was used in the discussion of the request           |
| oppose_symbol_count   | count how often a oppose symbol was used in the discussion of the request             |
| answer_symbol_count   | count how often a answer symbol was used in the discussion of the request             |
| support_symbol_count  | count how often a support symbol was used in the discussion of the request            |
| html                  | raw html of the site that can be found under the url. only the main part of the site is saved. |
| task                  | short description of the bot function                                                 |
| code                  | link to the online repository or other code describing text                           |
| function              | detailed description of the bot function                                              |
| archive_comment       | short comment why the request for permission was successful or not. Example can be found under the following link: https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/April_2013 (see for example '(approved temporary)') |
| summary               | summary about the discussion and the final decision about the request for permission  | 
| retrieved_at          | timestamp when the data was retrived                                          |

The tabel `bots` contains the following content:
Note: The lates update for this table was on 17 July 2018 and thus may not be up-to-date!


| column                | content explanation                                                                   |
| ----------------------|:-------------------------------------------------------------------------------------:|
| id                    | unique identifier |
| userid                | user id on wikidata |
| name                  | name of the bot                                                                       |
| has_botflag           | flag which states if the bot has a bot flag. 1 if it has and 0 if not |
| is_extension_bot      | flag which states if the bot is a extension bot. 1 if it is and 0 if not|
| is_blocked            | flag which states if the bot is blocked. 1 if it is and 0 if not|
| blockid               | ... |
| blockedby             | ... |
| blockedbyid           | ... |
| blockedtimestamp      | timestamp when the bot was blocked |
| blockreason           | reason why the bot was blocked |
| blockexpiry           | ... |
| groups                | groups which the bot belongs to |
| implicitgroups        | implicit groups which the bot belongs to|
| rights                | rights which the bot has |
| editcount             | count of edits the bot has made so far |
| registration          | timestamp when the bot was registered |
| redirect_to           | if the bot name is an alias of another name, the id of the other bot is mentioned here |
| retrieved_at          | timestamp when the data was retrived |

## Analysis

### Analysis of Wikidata Bots

This chapter aims to analyse the bots on Wikidata. 

### Number of bots with respect to there origins
As a first step we try to find out the number of bots with respect to there origins. Note: The list is updateed manually thus disparities between this tables and the following results may occur! Last update: 17 July 2018

| criteria                           | amount       | 
| ---------------------------------- |:------------:|
| number of users in the group 'bot' | 252 |
| number of bots that have a closed request for permission | 363 |
| number of bots with a botflag     | 259 |
| number of bots without a botflag  | 164 |
| number of extension bots           | 6 |

The following statistics describe partially the same as the table before but instead of fetching the results from the csv files, they are retrived from the db and contain more crtieria. The calcuated values are summarizes in the subsequent matrix. An example of how to interpret the matrix is that the field `b_f ex_b` describe how many distinct bots which have a bot flag and which are extension bots exist. If you want to see the names of the bots which belong to each group, you can uncomment the line ``` Analyser.print_matrix_bot_names() ```. The column names are described as follows:

| column name               | explanation                               |
| --------------------------|:-----------------------------------------:|
| b_f                       | has bot flag                              |
| no_b_f                    | has no bot flag                           |
| ex_b                      | is a extension bot                        |
| group                     | is in group 'bot'                         |
| request                   | has a request for permission              |
| s_request                 | has a successful request for permission   |
| u_request                 | has a unsuccessful request for permission |


In [22]:
Analyser.generate_matrix()

           b_f no_b_f ex_b group request s_request u_request
b_f        259      0    0   202     249       249        20
no_b_f     NaN    164    0    23      63        41        24
ex_b       NaN    NaN    6     3       1         1         0
group      NaN    NaN  NaN   252     242       242        15
request    NaN    NaN  NaN   NaN     363       317        70
s_request  NaN    NaN  NaN   NaN     NaN       317        24
u_request  NaN    NaN  NaN   NaN     NaN       NaN        70


In [21]:
# Analyser.print_matrix_bot_names()

### Number and names of unique bots

The following describes the number and the names of all unique bots. These bots are retrieved from all lists (e.g. bots with a bot flag, bots without a bot flag, bots with a request for permission, bots in the user group 'bot' and extension bots)

In [3]:
# Analyser.print_unique_bots()

### Differences between unique bots with a request and bots without a request
The following section compares all unique bots which have a request for permission and those who don't.

#### Number and names of bots with a request for permission.

In [34]:
# Analyser.print_bots_with_request()

#### Number and names of bots without a request for permission.

In [35]:
# Analyser.print_bots_without_request()

#### Rights of bots with a request for permission.
The following shows how many bots with a request for permission have a specific right. Note: the rights could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 357 and not 363!

In [37]:
# Analyser.print_rights_of_bots_with_request()

#### Bots with request without rights in the database
The following tries to answer the question, why there are bots in our database for which no rights could be retrieved from the api.

In [39]:
# Analyser.print_bots_with_request_without_rights()

All bots for which no rights could be retrived and which have a red link, were probably deleted and therefore have no user account for which rights can be retrived.

The bot 'Global Economic Map Bot' has its own user page but still no rights can be retrived; it is not yet solved why (TODO).

Note: There can alo be a discrepancy between the total amount of bots with a request (e.g. 368 at the time of 17. July 2018) and the sum of bots with rights in the db (e.g. 362) plus bots with no rights and no redlink (e.g. 1) and bots with a red link (e.g. 5). This can happen if various requests for permissions for one bot exist and for one request only a red link for the bot could be parsed and for the other a working link could be parsed. 

#### Rights of bots without a request for permission.
The following shows which rights bots without a request for permission have and how many of them have these right. Note: the rights could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 117 and not 119!

In [50]:
# Analyser.print_rights_of_bots_without_request()

#### Bots without request and without rights in the database

The following tries to answer the question, why there are bots in our database for which no rights could be retrieved from the api.

In [43]:
# Analyser.print_bots_without_request_without_rights()

The bot '127.0.0.1' is no user in the common sense. It belongs to the extension bots and is part of the MediaWiki software. Therefore no rights can be parsed.

For the bot 'Eflybot' rights can only be retrieved over the MediaWikiApi (e.g. https://www.mediawiki.org/w/api.php) and not the Wikdata API (e.g. https://www.wikidata.org/w/api.php). It is not yet solved why (TODO).

#### Differences of bot rights
The following shows all right that bots with a request for permission have but all other bots do not have and vice versa.

In [45]:
# Analyser.print_right_differences_for_request()

#### Groups of bots with a request for permission.
The following shows how many bots with a request for permission belong to a specific group. Note: the groups could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 357 and not 363!

Note: All visitors to the site, including unregistered users, are part of the '*' group and get associated rights (for more infromation see: https://en.wikipedia.org/wiki/Wikipedia:User_access_levels#User_groups)

In [50]:
# Analyser.print_groups_of_bots_with_request()

#### Bots with request and without groups in db
The following tries to answer the question, why there are bots in our db for which no groups could be retrieved from the api.

In [52]:
# Analyser.print_bots_with_request_without_groups()

All bots for which no groups could be retrived and which have a red link, were probably deleted and therefore have no user account for which groups can be retrived.

The bot 'Global Economic Map Bot' has its own user page but still no groups can be retrived; it is not yet solved why (TODO).

Note: There can alo be a discrepancy between the total amount of bots with a request (e.g. 368 at the time of 17. July 2018) and the sum of bots with groups in the db (e.g. 362) plus bots with no groups and no redlink (e.g. 1) and bots with no groups and a red link (e.g. 5). This can happen if various requests for permissions for one bot exist and for one request only a red link for the bot could be parsed and for the other a working link could be parsed. 

#### Groups of bots without a request for permission.
The following shows how many bots without a request for permission belong to a specific group. Note: the groups could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 117 and not 119!

In [54]:
# Analyser.print_groups_of_bots_without_request()

#### Bots with request and without groups in db
The following tries to answer the question, why there are bots in our db for which no groups could be retrieved from the api.

In [56]:
# Analyser.print_bots_without_request_without_groups()

The bot '127.0.0.1' is no user in the common sense. It belongs to the extension bots and is part of the MediaWiki software. Therefore no groups can be parsed.

For the bot 'Eflybot' groups can only be retrieved over the MediaWikiApi (e.g. https://www.mediawiki.org/w/api.php) and not the Wikdata API (e.g. https://www.wikidata.org/w/api.php). It is not yet solved why (TODO).

#### Differences of bot groups
The following shows all groups that bots with a request for permission belong to but all other bots do not and vice versa.

In [58]:
# Analyser.print_groups_differences()

### Differences between bots with bot flag and bots in the group bot
The following section compares all unique bots which have a bot flag (and do not belong to the group bot) and bots which belong to the group bot (and do not have a bot flag).
#### Number and names of bots with a bot flag and do not belong to the group bot

In [60]:
# Analyser.print_bots_with_bot_flag()

#### Number and names of bots which belong to the group bot and do not have a bot flag

In [56]:
# Analyser.print_bots_in_bot_group()

#### Number and names of bots with a bot flag and which belong to the group bot
Note: If you add up this number of bots and the number of bots which belong to the group bot and do not have a bot flag, you will not get the overall amount of bots which belong to the bot group. This effect is due to the fact that in the database bots exist which belong to the bot group but have a NULL value for the field `has_botflag` and therefore are not captured by one of these queries.

In [58]:
# Analyser.print_bots_with_bot_flag_and_in_bot_group()

#### Rights of bots with a bot flag

In [42]:
# Analyser.print_rights_of_bot_with_bot_flag()

#### Rights of bots which belong to the group 'bot'

In [46]:
# Analyser.print_right_differences_for_bot_flag_and_bot_group()

#### Differences of bot rights
The following shows all right that bots with a bot flag have but bots which belong to the group bot do not have and vice versa. 

In [50]:
# Analyser.print_right_differences_for_bot_flag_and_bot_group()

### Analysis of groups

In [125]:
# Analyser.plot_bots_group_destribution()

### Analysis of edit count

List of bots ordered by their edit count.

In [92]:
# Analyser.print_edit_count()

List of bots ordered by and their average edit count per day since there registration.

In [95]:
# Analyser.print_average_edit_count_per_day()

### Analysis of Requests for Permission

This chepter aims to analyse the requests for permissions for bots on Wikidata.

#### General characteristics

The following table contain some key figures about the requests for permissions in our database. As reference point the date when the request was first edited is taken. Note: The list is updateed manually thus disparities between this tables and the values in the database may occur! Last update: 24 July 2018

| criteria                       | amount                |
| -------------------------------|:---------------------:|
| number of bots                 |368                    |
| number of requests             |683                    |
| number of succesful bots       |322                    |
| number of succesful requests   |599                    |
| number of unsuccesful bots     |70                     |
| number of unsuccesful requests |84                     |
| date of the first request      |2012-11-13 03:29:25    |
| date of the last request       |2018-07-14 21:26:58    |

In [48]:
Analyser.plot_general_statistics_about_requests()

#### Distribution of request for permission

This shows the distribution of request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [41]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE closed_at <> ''
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Distribution of successful request for permission

This shows the distribution of successful request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [42]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE is_successful = 1 
    AND closed_at <> '' 
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Distribution of unsuccessful request for permission

This shows the distribution of unsuccessful request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [43]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE is_successful = 0 
    AND closed_at <> ''
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Requests for permissions where closed_at is not set

TODO: Check why that happens.

In [123]:
# Analyser.print_request_for_permission_without_closed_at()

#### Distribution of editor count

This shows the distribution of editor count per request for permission. The editor count represents the number of different users who edited the request for permission.

In [46]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    GROUP BY editor_count
    ''')

List urls of requests for permissions ordered by their editor count.

In [105]:
# Analyser.print_editor_count()

#### Distribution of editor count with a successful request for permission

In [45]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    AND is_successful = 1
    GROUP BY editor_count
    ''')

List urls of succesful requests for permissions ordered by their editor count.

In [107]:
# Analyser.print_editor_count(mode='successful')

In [44]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    AND is_successful = 0
    GROUP BY editor_count
    ''')

List urls of succesful requests for permissions ordered by their editor count.

In [110]:
# Analyser.print_editor_count(mode='unsuccessful')

#### Requests for permissions where editor_count is not set

TODO: Check why that happens.

In [62]:
# Analyser.print_request_for_permission_without_editor_count()